함수 데커레이터는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해준다. 이를 위해 다음을 살펴본다.

* 파이썬이 데커레이터 구문을 평가하는 방식
* 변수가 지역 변수인지 파이썬이 판단하는 방식
* 클로저의 존재 이유와 작동 방식
* `nonlocal`로 해결할 수 있는 문제

이후 다음과 같은 주제를 다룬다.

* 잘 작동하는 데커레이터 구현하기
* 표준 라이브러리에서 제공하는 재미있는 데커레이터들
* 매개변수화된 데커레이터 구현하기

# 7.1 데커레이터 기본 지식

데커레이터는 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)이다. 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다. 데커레이터는 다음과 같이 작동한다.

```python
@decorate
def target():
    print('running target()')
```

위 코드는 다음 코드와 동일하게 작동한다.

```python
def target():
    print('running target()')
    
target = decorate(target)
```

두 코드를 실행한 후 `target`은 원래의 `target()` 함수를 가리키는 것이 아니라 `decorate(target)`이 반환한 함수를 가리키게 된다.

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [5]:
@deco
def target():
    print("running target()")
    
target() # 실제로는 inner()를 실행한다.

running inner()


In [8]:
target # inner()를 가리키고 있음

<function __main__.deco.<locals>.inner()>

요약하면, 
* 데커레이터는 데커레이트된 함수를 다른 함수로 대체한다.
* 데커레이터는 모듈이 로딩될 때 바로 실행된다.

# 7.2 파이썬이 데커레이터를 실행하는 시점

데커레이터의 핵심 특징은 데커레이트된 함수가 정의된 직후, 즉 일반적으로 파이썬이 모듈을 로딩하는 시점에 실행된다는 것이다.

In [15]:
!cat registration.py

registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1() # 이 경우 register()가 원래 함수를 반환하므로 f1()이 실행됨
    f2()
    f3()

if __name__=='__main__':
    main()


In [14]:
!python registration.py

running register(<function f1 at 0x7f4e78a3c0d0>)
running register(<function f2 at 0x7f4e76b6d6a8>)
running main()
registry -> [<function f1 at 0x7f4e78a3c0d0>, <function f2 at 0x7f4e76b6d6a8>]
running f1()
running f2()
running f3()


`register()`는 모듈 내의 다른 어떠한 함수보다 먼저 실행된다. 모듈이 로딩된 후 `registry`는 데커레이트된 두 개의 함수 `f1()`과 `f2()`에 대한 참조를 가진다. 이 두 함수는 `main()`에 의해 명시적으로 호출될 때만 실행된다.

In [18]:
import registration # 임포트 순간에 데커레이터가 실행됨

In [17]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

이 예제는 `임포트 타임`과 `런타임`의 차이를 명확히 보여준다.

# 7.3 데커레이터로 개선한 전략 패턴

데커레이터는 6.1절에서 구현한 전자상거래 프로모션 할인 코드를 개선하는데 유용하게 사용할 수 있다.

전자상거래 예제의 가장 큰 문제는 함수를 정의할 때, 그리고 가장 큰 할인 방식을 결정하는 `best_promo()` 함수에 의해 사용되는 `promos` 리스트에 함수명을 반복해서 사용한다는 점이다. 이를 데커레이터를 이용해서 다음과 같이 해결할 수 있다.

In [21]:
promos = []

def promotion(promo_func):
    """promotion() 데커레이터는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

이 방법은 다음과 같은 장점을 가진다.

* 프로모션 전략 함수명이 특별한 형태로 되어 있을 필요가 없다.
* `@promotion` 데커레이터는 임시로 어떤 프로모션을 배제할 수 있다.
* 프로모션 할인 전략을 구현한 함수는 `@promotion` 데커레이터가 적용되는 한 어느 모듈에서든 정의할 수 있다.

# 7.4 변수 범위 규칙

다음 예제에서는 함수 매개변수로 정의된 지역 변수 a와 함수 내부에 정의되지 않은 변수 b 등 두 개의 변수를 읽는 함수를 정의하고 테스트한다.

In [22]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [23]:
b=6
f1(3)

3
6


In [26]:
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

이는 파이썬이 함수 본체를 컴파일할 때 b가 함수 안에서 할당되므로 b를 지역 변수로 판단하기 때문이다. 함수 안에 할당하는 문장이 있지만 인터프리터가 b를 전역 변수로 다루기 원한다면 `global` 키워드를 이용하여 선언해야 한다.

In [30]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [31]:
b

9

In [32]:
f3(3)

3
9


In [33]:
b=30
f3(3)

3
30


`dis` 모듈을 사용하면 파이썬 함수를 쉽게 바이트코드로 디스어셈블할 수 있다.

In [34]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [36]:
dis(f2) # b를 지역 변수에서 불러옴

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# 7.5 클로저

클로저는 함수 본체에서 정의하지 않고 참조하는 비전역(nonglobal) 변수를 포함한 확장 범위를 가진 함수다. 예제를 통해 알아보자.

In [37]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [38]:
avg = Averager()
avg(10)

10.0

In [39]:
avg(11)

10.5

In [40]:
avg(12)

11.0

고위 함수 `make_averager()`를 이용해서 구현한 예제를 보자.

In [44]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [45]:
avg = make_averager()
avg(10)

10.0

In [47]:
avg(11)

10.5

In [48]:
avg(12)

11.0

두번째 예제의 `avg()` 함수는 어디에서 `series`를 찾을까? `average` 안에 있는 `series`는 자유 변수(free variable)이다. 이는 지역 범위에 바인딩되어 있지 않은 변수라는 의미이다.

In [49]:
avg.__code__.co_varnames

('new_value', 'total')

In [50]:
avg.__code__.co_freevars

('series',)

`series`에 대한 바인딩은 `avg()` 함수의 `__closure__` 속성에 저장되며, 이는 `avg.__code__.co_freevars`의 이름에 대응된다. 이 항목은 `cell` 객체이며, 이 객체의 `cee_contents` 속성에서 실제값을 찾을 수 있다.

In [51]:
avg.__closure__

(<cell at 0x7ff9fe8b5378: list object at 0x7ff9ff514708>,)

In [52]:
avg.__closure__[0].cell_contents

[10, 11, 12]

클로저는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수다. 따라서 함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근할 수 있다.

함수가 '비전역' 외부 변수를 다루는 경우는 그 함수가 다른 함수 안에 정의된 경우 뿐이다.

# 7.6 nonlocal 선언

앞서 구현한 `make_averager()`는 매번 합계를 재계산하므로 효율적이지 않다. 이를 개선한 다음 함수는 어디가 잘못되었을까?

In [54]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [57]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

`averager()` 본체 안에서 `count` 변수에 할당하고 있으므로 `count`를 지역 변수로 만들어 문제가 발생한다. `series`는 가변형 리스트였으므로 문제가 없었다. 그러나 `count = count + 1`과 같은 문장으로 변수를 다시 바인딩하면 암묵적으로 `count`라는 지역 변수를 만든다. `count`가 더이상 자유 변수가 아니므로 클로저에 저장되지 않는다.

이를 해결하기 위해 `nonlocal` 선언이 소개되었다.

In [64]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [65]:
avg = make_averager()
avg(10)

10.0

# 7.7